# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:
* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM Schema

In almost all cases, rather than just working with a single table you will typically need data from multiple tables. 
Doing this requires the use of **joins** using shared columns from the two tables. 

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
# Your code here
import sqlite3
import pandas as pd
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

In [5]:
cur.execute('''SELECT * FROM employees
               LIMIT 5;''')
df = pd.DataFrame(cur.fetchall()) 
df.columns = [i[0] for i in cur.description]
df.head()

,employeeNumber,lastName,firstName,extension,email,officeCode,reportsTo,jobTitle
0,1002,Murphy,Diane,x5800,dmurphy@classicmodelcars.com,1,,President
1,1056,Patterson,Mary,x4611,mpatterso@classicmodelcars.com,1,1002,VP Sales
2,1076,Firrelli,Jeff,x9273,jfirrelli@classicmodelcars.com,1,1002,VP Marketing
3,1088,Patterson,William,x4871,wpatterson@classicmodelcars.com,6,1056,Sales Manager (APAC)
4,1102,Bondur,Gerard,x5408,gbondur@classicmodelcars.com,4,1056,Sale Manager (EMEA)


## Display the names of all the employees in Boston 

Hint: join the employees and offices tables.

In [6]:
# Your code here
cur.execute('''SELECT firstName, lastName FROM employees
            JOIN offices
            ON employees.officeCode = offices.officeCode
            WHERE city = 'Boston';''')
df = pd.DataFrame(cur.fetchall()) 
df.columns = [i[0] for i in cur.description]
df.head()

,firstName,lastName
0,Julie,Firrelli
1,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by.

In [8]:
# Your code here
cur.execute('''SELECT o.officeCode, o.city, 
            COUNT(e.employeeNumber) AS n_employees
            FROM offices AS o
            LEFT JOIN employees AS e
            USING(officeCode)
            GROUP BY officeCode;''')
df = pd.DataFrame(cur.fetchall()) 
df.columns = [i[0] for i in cur.description]
df.head(20)

# cur.execute("""SELECT o.officeCode, o.city, COUNT(e.employeeNumber) AS n_employees
#                FROM offices AS o 
#                LEFT JOIN employees AS e
#                USING(officeCode)
#                GROUP BY officeCode
#                HAVING n_employees = 0;""")
# df = pd.DataFrame(cur.fetchall())
# df.columns = [i[0] for i in cur.description]
# df

,officeCode,city,n_employees
0,1,San Francisco,6
1,2,Boston,2
2,3,NYC,2
3,4,Paris,5
4,5,Tokyo,2
5,6,Sydney,4
6,7,London,2
7,27,Boston,0


## Write 3 Questions of your own and answer them

In [ ]:
# Answers will vary
# Example: Display the htmlDescription and employee's first and last name for each product that each employee has sold
# What City has the most sales?
# Who is the best salesman?
# What is the most expensive product?

In [28]:
cur.execute("""SELECT officeCode, of.city, COUNT(productName)
               AS total_sold
               FROM employees e
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN orders o
               USING(customerNumber)
               JOIN orderdetails od
               USING(orderNumber)
               JOIN products p
               USING(productCode)
               JOIN offices of
               USING(officeCode)
               GROUP BY of.city
               ORDER BY total_sold DESC
               """)
               
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df))
display(df.head(20))
#officeCode, of.city, COUNT(productName)
#AS total_sold

7


,officeCode,city,total_sold
0,4,Paris,959
1,7,London,456
2,1,San Francisco,445
3,6,Sydney,370
4,3,NYC,353
5,2,Boston,276
6,5,Tokyo,137


In [31]:
cur.execute("""SELECT firstName, lastName, COUNT(productName)
               AS total_sold
               FROM employees e
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN orders o
               USING(customerNumber)
               JOIN orderdetails od
               USING(orderNumber)
               JOIN products p
               USING(productCode)
               GROUP BY lastName
               ORDER BY total_sold DESC
               LIMIT 1""")
               
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df))
display(df)


1


,firstName,lastName,total_sold
0,Gerard,Hernandez,396


In [32]:
# Your code here
cur.execute('''SELECT productName, productCode, buyPrice, MSRP 
               FROM products
               ORDER BY MSRP DESC LIMIT 1;''')
df = pd.DataFrame(cur.fetchall()) 
df.columns = [i[0] for i in cur.description]
df

,productName,productCode,buyPrice,MSRP
0,1952 Alpine Renault 1300,S10_1949,98.58,214.3


## Level Up: Display the names of every individual product that each employee has sold

In [15]:
# Your code here
cur.execute("""SELECT firstName, lastName, productName
               FROM employees e
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN orders o
               USING(customerNumber)
               JOIN orderdetails od
               USING(orderNumber)
               JOIN products p
               USING(productCode)
               """)
               
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df))
display(df.head(20))
df.tail()
# firstName, lastName, productName

2996


,firstName,lastName,productName
0,Leslie,Jennings,1958 Setra Bus
1,Leslie,Jennings,1940 Ford Pickup Truck
2,Leslie,Jennings,1939 Cadillac Limousine
3,Leslie,Jennings,1996 Peterbilt 379 Stake Bed with Outrigger
4,Leslie,Jennings,1968 Ford Mustang
5,Leslie,Jennings,1968 Dodge Charger
6,Leslie,Jennings,1970 Plymouth Hemi Cuda
7,Leslie,Jennings,1969 Dodge Charger
8,Leslie,Jennings,1948 Porsche 356-A Roadster
9,Leslie,Jennings,1969 Dodge Super Bee


,firstName,lastName,productName
2991,Martin,Gerard,1954 Greyhound Scenicruiser
2992,Martin,Gerard,1950's Chicago Surface Lines Streetcar
2993,Martin,Gerard,Diamond T620 Semi-Skirted Tanker
2994,Martin,Gerard,1911 Ford Town Car
2995,Martin,Gerard,1936 Mercedes Benz 500k Roadster


## Level Up: Display the Number of Products each employee has sold

In [29]:
cur.execute("""SELECT firstName, lastName, COUNT(productName)
               AS total_sold
               FROM employees e
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN orders o
               USING(customerNumber)
               JOIN orderdetails od
               USING(orderNumber)
               JOIN products p
               USING(productCode)
               GROUP BY lastName
               ORDER BY total_sold DESC
               """)
               
df = pd.DataFrame(cur.fetchall())
df.columns = [i[0] for i in cur.description]
print(len(df))
display(df)


15


,firstName,lastName,total_sold
0,Gerard,Hernandez,396
1,Leslie,Jennings,331
2,Pamela,Castillo,272
3,Larry,Bott,236
4,Barry,Jones,220
5,George,Vanauf,211
6,Andy,Fixter,185
7,Peter,Marsh,185
8,Loui,Bondur,177
9,Steve,Patterson,152


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!